# CNN LSTM

**Reference:** https://github.com/vinayakumarr/Network-Intrusion-Detection

**Dataset:** http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [1]:
import numpy as np
np.random.seed(1337)  # for reproducibility

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, MaxPooling1D, LSTM

from sklearn.preprocessing import Normalizer

In [2]:
DATA = '../data/'
MODELS = '../models/'
TEST = 0.3

In [3]:
traindata = pd.read_csv(DATA+'kddcup.data.corrected.csv', header=None,nrows=3000000)
traindata

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,smurf.
2999996,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,smurf.
2999997,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,smurf.
2999998,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,smurf.


In [4]:
from utils import preprocess
traindata,testdata=preprocess.preprocess(traindata)

In [5]:
class_to_idx={d:i for i,d in enumerate(traindata['Y'].unique())}
traindata['Y']=traindata['Y'].replace(class_to_idx)
testdata['Y']=testdata['Y'].replace(class_to_idx)

In [6]:
X = traindata.drop(columns=['Y'])
Y = traindata['Y']
C = testdata['Y']
T = testdata.drop(columns=['Y'])

In [7]:
scaler = Normalizer().fit(X)
X_train = scaler.transform(X)
X_test = scaler.transform(T)

In [8]:
y_train=Y
y_test= C

In [9]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [10]:
model = Sequential()
model.add(Conv1D(8, 5, padding="same", activation="tanh", input_shape=X_train.shape[1:]))
model.add(MaxPooling1D(2))
model.add(LSTM(8))
model.add(Dropout(0.1))
model.add(Dense(23, activation="softmax"))

In [11]:
# define optimizer and objective, compile model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 41, 8)             48        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 20, 8)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 8)                 544       
_________________________________________________________________
dropout (Dropout)            (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 23)                207       
Total params: 799
Trainable params: 799
Non-trainable params: 0
_________________________________________________________________


In [13]:
# train
model.fit(X_train, y_train, epochs=5, validation_split=0.25, shuffle=True, batch_size=512)

Epoch 1/5
3296/3296 [==============================] - 87s 26ms/step - loss: 0.5857 - accuracy: 0.7999 - val_loss: 0.0242 - val_accuracy: 0.9999
Epoch 2/5
3296/3296 [==============================] - 86s 26ms/step - loss: 0.0717 - accuracy: 0.9833 - val_loss: 0.0016 - val_accuracy: 0.9996
Epoch 3/5
3296/3296 [==============================] - 85s 26ms/step - loss: 0.0439 - accuracy: 0.9893 - val_loss: 9.5028e-04 - val_accuracy: 0.9997
Epoch 4/5
3296/3296 [==============================] - 84s 25ms/step - loss: 0.0351 - accuracy: 0.9922 - val_loss: 0.0010 - val_accuracy: 0.9996
Epoch 5/5
3296/3296 [==============================] - 85s 26ms/step - loss: 0.0301 - accuracy: 0.9931 - val_loss: 5.0212e-04 - val_accuracy: 0.9997


In [14]:
model.evaluate(X_test, y_test)

23438/23438 [==============================] - 29s 1ms/step - loss: 6.6757e-06 - accuracy: 1.0000


[6.675719760096399e-06, 1.0]

In [15]:
model.save(MODELS+"cnn_lstm_model.hdf5")